In [1]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(os.path.join(project_root, 'code'))

from model.dataset import MODMA
from model.run_train import run
import model.config as config

import warnings 
warnings.filterwarnings("ignore")

import torch
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, classification_report
from transformers import AutoTokenizer
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import time
from pathlib import Path


/home/zhhaibo2023/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 训练模型

In [2]:
SEED = 2025
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [3]:
device = 'cuda:0'

# 加载数据
r_data = torch.load('../data/modma.pt')
tokenizer = AutoTokenizer.from_pretrained(config.ptm)  # 使用BERT中文分词器

dataset_name = 'modma'
dataset = MODMA(
    dataset_name = dataset_name,
    response_texts=r_data['responses'],
    labels=r_data['labels'],
    tokenizer=tokenizer,
    max_length=64
)
print(len(dataset))

52


In [4]:
LR = 5e-5
Weight_Decay = 1e-4
EPOCHS = 50
BS = 16
Dropout = 0.5


gammas = [1.0, 0.7072, 0.2005]

classify_flag ='bin' # penta

logdir_root = os.path.join(os.getcwd(), dataset_name)
print(logdir_root)

/home/zhhaibo2023/program/msce/code/run_model/modma


In [9]:
test_label_list_perfold, test_pred_list_perfold, test_id_list_perfold, best_test_acc, best_test_fs = run(dataset=dataset,
                                                                                                        dataset_name=dataset_name,
                                                                                                        logdir_root=logdir_root,
                                                                                                        classify_flag=classify_flag,
                                                                                                        device=device,
                                                                                                        gammas=gammas,
                                                                                                        SEED=SEED,
                                                                                                        BS=BS,
                                                                                                        LR=LR,
                                                                                                        Weight_Decay=Weight_Decay,
                                                                                                        Dropout=Dropout,
                                                                                                        EPOCHS=EPOCHS,
                                                                                                        K=5)

bert_model_path: /home/zhhaibo2023/Pretrained_Model/bert-base-chinese


Model: DepressionDetectionModel
num_stimuls:  18
fold: 1, epoch: 1, train_loss: 3.2855, train_acc: 43.9, train_fscore: 34.29, test_loss: 3.1263, test_acc: 54.55, test_fscore: 0.0, time: 2.61 sec
fold: 1, epoch: 2, train_loss: 2.8693, train_acc: 56.1, train_fscore: 0.0, test_loss: 2.4699, test_acc: 54.55, test_fscore: 0.0, time: 2.12 sec
fold: 1, epoch: 3, train_loss: 2.373, train_acc: 39.02, train_fscore: 41.86, test_loss: 1.9379, test_acc: 72.73, test_fscore: 76.92, time: 2.24 sec
fold: 1, epoch: 4, train_loss: 1.8711, train_acc: 63.41, train_fscore: 51.61, test_loss: 1.7456, test_acc: 54.55, test_fscore: 0.0, time: 2.17 sec
fold: 1, epoch: 5, train_loss: 1.7311, train_acc: 58.54, train_fscore: 10.53, test_loss: 1.5757, test_acc: 81.82, test_fscore: 75.0, time: 2.26 sec
fold: 1, epoch: 6, train_loss: 1.5842, train_acc: 63.41, train_fscore: 28.57, test_loss: 1.4859, test_acc: 54.55, test_fscore: 0.0, time: 2.19 sec
fold: 1, epoch: 7, train_loss: 1.5141, train_acc: 70.73, train_fscore: 